In [1]:
FILE_PATH = "/home/choice/Desktop/insurance-fullstack/data/SBI historical data.csv"

In [2]:
import pandas as pd

df = pd.read_csv(FILE_PATH)

In [3]:
df = df.drop(columns=['Unnamed: 0', 'System Projected Revenue'])

In [4]:
df.head(5)

,Policy Id,Name of Insurance Company,Third Party Premium,OD Premium,Net Premium,Total Premium,Tax,Product Name,Product Category,Registration Number,Vehical Type,CC,Petrol Type,Model,Make,RTO Location,Vehical Category,Projected Revenue
0,137449,SBI General Insurance,2144.0,669.0,3138.0,3703.0,565.0,Private Car Package Policy,Private Car Package Policy,KA68M1034,NaN,998.0,Petrol,ALTO VXI,MARUTI SUZUKI,Kolar,Private Car,173.94
1,137479,SBI General Insurance,3466.0,720.0,4511.0,5323.0,812.0,Private Car Package Policy,Private Car Package Policy,KA50P5921,NaN,1196.0,Petrol,EECO,MARUTI SUZUKI,Bengaluru,Private Car,201.60
2,137487,SBI General Insurance,2144.0,518.0,2987.0,3525.0,538.0,Private Car Package Policy,Private Car Package Policy,KA48M7843,NaN,793.0,Petrol,CELERIO,MARUTI SUZUKI,Bagalkot,Private Car,134.68
3,137497,SBI General Insurance,5773.0,185.0,6283.0,7414.0,1131.0,THREE WHEELER PCV PACKAGE POLICY,3W PCV Package Policy,AP31TG4805,3W PCV,436.0,Petrol,ALFA,MAHINDRA & MAHINDRA,Visakhapatnam,Commercial,3644.14
4,136707,SBI General Insurance,2094.0,0.0,2094.0,2471.0,377.0,Private Car Liability Only Policy,Private Car Liability Only Policy,DL09CW9506,NaN,998.0,Petrol,Alto K10,MARUTI SUZUKI,New Delhi,Private Car,774.78


In [5]:
df.isnull().sum()

Policy Id                     0
Name of Insurance Company     0
Third Party Premium           0
OD Premium                    0
Net Premium                   0
Total Premium                 0
Tax                           0
Product Name                  0
Product Category              0
Registration Number           0
Vehical Type                 56
CC                            8
Petrol Type                   0
Model                         0
Make                          0
RTO Location                  0
Vehical Category              0
Projected Revenue             0
dtype: int64

In [6]:
query = {
    "Policy Type": None, 
    # "Product Name": None, 
    # "Product Category": None,
    "State": None, 
    "City": None, 
    "Vehicle Category": None, 
    "Vehicle Type": None, 
    "Model": None, 
    "Make": None,
    "Fuel Type": None, 
    "Power": None, 
}

In [7]:
def get_state_from_rto_code(rto_code: str) -> str:
    """
    Description:
        - This function gets state name from the rto code

    Parameters:
        - rto_code (str): RTO Code of the the state
    
    Returns:
        - state (str): State corresponding to the RTO Code
    """

    with open("/home/choice/Desktop/insurance-fullstack/data/states.json", 'r') as jsonFile:
        rto_code_data = json.load(jsonFile)

    for state in rto_code_data:
        if state['RTO_CODE'] == rto_code:
            state = state['State']
            return state

In [8]:
from typing import Dict
import json
def generate_query(row: pd.Series) -> Dict:
    """
    Description:
        - This function takes a row of the dataframe and generates a structured query out of it.

    Parameters:
        - row (pandas series): Row for which we want to generate a query for 

    Returns:
        - query (obj): A structured query object
    """

    # Defining the query object
    query = {
        "Policy Type": None, 
        # "Product Name": None, 
        # "Product Category": None,
        "State": None, 
        "City": None, 
        "Vehicle Category": None, 
        "Vehicle Type": None, 
        "Model": None, 
        "Make": None,
        "Fuel Type": None, 
        "Power": None, 
    }

    # Condition for Policy Type
    
    if row['Third Party Premium'] == 0.0 and row['OD Premium'] > 0.0:
        query["Policy Type"] = "OD/SAOD/SOD/1+0"
    elif row['Third Party Premium'] > 0.0 and row['OD Premium'] == 0.0:
        query['Policy Type'] = "TP/ACT/SATP/STP/0+1/AOTP"
    elif row['Third Party Premium'] > 0.0 and row['OD Premium'] > 0.0:
        query['Policy Type'] = "Comp/Package/Bundle/Comprehensive"

    # Getting state from rto code
    registration_number = row['Registration Number']

    state_code = registration_number[:2]

    state_name = get_state_from_rto_code(state_code)

    query['State'] = f"{state_name} ({state_code})"

    # RTO City
    query['City'] = row['RTO Location']

    # Vehicle Category
    query['Vehicle Category'] = row['Vehical Category']

    # Vehicle Type
    if row['Vehical Type'] is np.nan:
        query['Vehicle Type'] = ""
    else:
        query['Vehicle Type'] = row['Vehical Type']

    # Model
    query['Model'] = row['Model']

    # Make
    query['Make'] = row['Make']

    # Fuel Type
    query['Fuel Type'] = row['Petrol Type']

    # Power
    query['Power'] = f"{row['CC']}cc"

    return query



In [9]:
import numpy as np
from tree import Tree
from dotenv import load_dotenv
import json
load_dotenv()

JSON_FILE_PATH = "/home/choice/Desktop/insurance-fullstack/data/hierarchichal json/hierarchical_data_SBI.json"

# Loading our tree

i = 0
results = []
for index, rows in df.iterrows():
    query = generate_query(rows)

    print(query)

    tree = Tree(file_path=JSON_FILE_PATH)
    try:
        best_keys_path, final, commission = tree.get_commision_rate(query)

        net_premium = rows['Net Premium']
        total_premium = rows['Total Premium']
        tax = rows['Tax']
        expected_revenue = rows['Projected Revenue']

        # Premium values 
        net_premium_revenue = None
        od_revenue = None
        tp_revenue = None
        # print("### PATH ###")
        # print(f"Best key path: {best_keys_path}")
        # print("Suggestions: ", list(final.keys()))

        # print("## COMMISSION VALUES ##")
        # print(f"System Commission: {commission}%")
        # print(f"Expected Revenue: {expected_revenue}")

        # Revenue on OD/TP
        if rows['Third Party Premium'] == 0.0 and rows['OD Premium'] > 0.0:
            net_premium_revenue = net_premium * (commission / 100)
            od_revenue = net_premium * (commission / 100)
            tp_revenue = 0
            # print(f"Revenue on Net Premium (OD):  {net_premium * (commission / 100)}")
            # print(f"Revenue on OD: {rows['OD Premium'] * (commission / 100)}")
        elif rows['Third Party Premium'] > 0.0 and rows['OD Premium'] == 0.0:
            net_premium_revenue = net_premium * (commission / 100)
            od_revenue = 0
            tp_revenue = rows['Third Party Premium'] * (commission / 100)
            # print(f"Revenue on Net Premium (TP): {net_premium * (commission / 100)}")
            # print(f"Revenue on TP: {rows['Third Party Premium'] * (commission / 100)}")
        elif rows['Third Party Premium'] > 0.0 and rows['OD Premium'] > 0.0:
            net_premium_revenue = net_premium * (commission / 100)
            od_revenue = rows['OD Premium'] * (commission / 100)
            tp_revenue = rows['Third Party Premium'] * (commission / 100)

            # print(f"Revenue on Net Premium: {net_premium * (commission / 100)}")
            # print(f"Revenue on TP: {rows['Third Party Premium'] * (commission / 100)}")
            # print(f"Revenue on OD: {rows['OD Premium'] * (commission / 100)}")

        results.append({
            "query": query, 
            "System Commission": commission,
            "Expected Revenue": expected_revenue, 
            "Net Premium Revenue": net_premium_revenue, 
            "TP Revenue": tp_revenue, 
            "OD Revenue": od_revenue, 
            "Path": best_keys_path, 
            "Other Suggestions": final
        })
        i += 1
        with open("results.json", 'w') as jsonFile:
            json.dump(results, jsonFile, indent=4)
        print(f"{i}/{df.shape[0]}")

        print("\n")
    except Exception as e:
        pass

{'Policy Type': 'Comp/Package/Bundle/Comprehensive', 'State': 'Karnataka (KA)', 'City': 'Kolar', 'Vehicle Category': 'Private Car', 'Vehicle Type': '', 'Model': 'ALTO VXI', 'Make': 'MARUTI SUZUKI', 'Fuel Type': 'Petrol', 'Power': '998.0cc'}
Best Key South 2
Best Key ROKarnataka
Best Key Bangalore
Best Key Pvt Car - Above 3 lakhs (Pvt car & 2W has common slabs)
Best Key Pvt Car (PO On OD) Comp & SAOD.
1/137


{'Policy Type': 'Comp/Package/Bundle/Comprehensive', 'State': 'Karnataka (KA)', 'City': 'Bengaluru', 'Vehicle Category': 'Private Car', 'Vehicle Type': '', 'Model': 'EECO', 'Make': 'MARUTI SUZUKI', 'Fuel Type': 'Petrol', 'Power': '1196.0cc'}
Best Key South 2
Best Key ROKarnataka
Best Key Bangalore
Best Key Pvt Car - Above 3 lakhs (Pvt car & 2W has common slabs)
Best Key Pvt Car (PO On OD) Comp & SAOD.
2/137


{'Policy Type': 'Comp/Package/Bundle/Comprehensive', 'State': 'Karnataka (KA)', 'City': 'Bagalkot', 'Vehicle Category': 'Private Car', 'Vehicle Type': '', 'Model': 'CELERIO', 

In [10]:
import json
with open("results.json", 'w') as jsonFile:
    json.dump(results, jsonFile, indent=4)

In [14]:
query_array = []
system_commission_array = []
expected_revenue_array = []
net_premium_revenue_array = []
tp_revenue_array = []
od_revenue_array = []
path_array = []
other_suggestions_array = []

for result in results:
    query_array.append(result['query'])
    system_commission_array.append(result['System Commission'])
    expected_revenue_array.append(result['Expected Revenue'])
    net_premium_revenue_array.append(result['Net Premium Revenue'])
    tp_revenue_array.append(result['TP Revenue'])
    od_revenue_array.append(result['OD Revenue'])
    path_array.append(result['Path'])
    other_suggestions_array.append(result['Other Suggestions'])

In [15]:
data = {
    "query": query_array, 
    "System Commission": system_commission_array, 
    "Expected Revenue": expected_revenue_array,
    "Net Premium Revenue": net_premium_revenue_array, 
    "TP Revenue": tp_revenue_array, 
    "OD Revenue": od_revenue_array, 
    "Path": path_array, 
    "Other Suggestions": other_suggestions_array
}

In [17]:
result_df = pd.DataFrame(data, columns=list(results[0].keys()))

In [ ]:
result_df.to_csv